In [1]:
import json
import numpy as np
import math
import os
import cv2
import open3d as o3d
import natsort 
import shutil
from pyquaternion import Quaternion
import sys
import random

output_path='/mnt/nas3/Data/PCD/indoor_parking/full/custom/'
asset_path ='/mnt/nas3/Data/PCD/indoor_parking/asset/'
label_path ='/mnt/nas3/Data/PCD/indoor_parking/labels/'


train_ratio = 0.9
class_names=[]
max_int = sys.maxsize

data_dir_num_thres = max_int

#Q : FOV아니고 whole scene label?
# TODO : float32저장, np.load, no label 삭제, label이 띄어쓰기는 _등으로 연결

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def convert_pcd_to_npy(file_path,save_path):
    point_cloud = o3d.read_point_cloud(file_path)
    pointcloud = np.asarray(point_cloud.points)[:, :3]
    zeros = np.zeros(np.shape(pointcloud)[0])
    zeros = np.reshape(zeros, (-1, 1))
    points = np.concatenate([pointcloud, zeros], axis=1)
    np.save(save_path, points)

def convert_bin_to_npy(file_path,save_path):
#     print(file_path)
    try:
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        np.save(save_path, points)
    except:
        print(file_path)
    

In [3]:
'''
move pcd file to each folder and rename as order
'''
def copy_pcd(mode,path,output_path,idx):       
    try:
        if not os.path.exists(output_path+'points/'):
            os.makedirs(output_path+'points/')
    except OSError:
        print("Error: Failed to create the directory.")
   
    if mode == "pcd":
        output_img_path = output_path+'points/'
        convert_pcd_to_npy(path, output_img_path+str(idx).zfill(6)+".npy")  
    elif mode == "bin":
        output_img_path = output_path+'points/'    
        convert_bin_to_npy(path, output_img_path+str(idx).zfill(6)+".npy")          



In [4]:
'''
convert label info to custom form

format: [x y z dx dy dz heading_angle category_name]

'''
def preprocess_label(label_output,frame_num):
    label_output = label_output+"labels/"
    try:
        if not os.path.exists(label_output):
            os.makedirs(label_output)
    except OSError:
        print("Error: Failed to create the directory.")
    for i in range(frame_num):
        try:
            f = open(label_output+str(label_idx+i).zfill(6)+".txt", 'w')
            f.close()
        except:
            print("Fail to create file")

def convert_label_suite2custom(label_path,label_output, label_idx): 
    label_output = label_output+"labels/"
    try:
        file = open(label_path)
    except:
        print("Error: Failed to open the file")


    jsonString = json.load(file)   
    # frame_num = jsonString.get("label").get('asset').get("info").get("frame_count")


    try:
        jsonArray = jsonString.get('objects')
    except:
        print("can't find objects")
        # return label_idx
        return
    
    # calib_path = output_path+'training/calib/'
    for iter_num, list in enumerate(jsonArray):
        frames = list.get("frames")
        class_name = list.get('class_name')
        if class_name == "Wall" or class_name == 'Undefined Object' or class_name =='Undefined Stuff':
            continue
#             class_name=="No Parking Sign"
        if class_name == "No Parking Sign":
            class_name = "No_Parking_Sign"
        if class_name == 'Traffic Cone':
            class_name = 'Traffic_Cone'
        if class_name =="Shopping Cart":
            class_name = 'Shopping_Cart'
        if class_name =="Tow-Wheeled Vehicle":
            class_name = 'Tow_Wheeled_Vehicle'
        if class_name =="Two-Wheeled Vehicle":
            class_name = 'Two_Wheeled_Vehicle'
        global class_names
        if (class_name in class_names) is not True:
            
            class_names.append(class_name)
            
        for frame in frames:
            idx = frame.get('num')
            
            ann = frame.get('annotation').get('coord')
            height = ann.get('size').get('z')
            width = ann.get('size').get('y')
            length = ann.get('size').get('x')
            # TODO : check coordinate , lidar or cam? -> cam
            
            x = ann.get('position').get('x')
            y = ann.get('position').get('y')
            z = ann.get('position').get('z')
           
#             yaw_pitch_roll = Quaternion(ann.get('rotation_quaternion').get('w'),ann.get('rotation_quaternion').get('x'),ann.get('rotation_quaternion').get('y'),ann.get('rotation_quaternion').get('z')).yaw_pitch_roll
            yaw_pitch_roll = Quaternion(
                ann.get('rotation_quaternion').get('w'),
                ann.get('rotation_quaternion').get('x'),
                ann.get('rotation_quaternion').get('y'),
                ann.get('rotation_quaternion').get('z'),
            ).yaw_pitch_roll
            yaw = yaw_pitch_roll[0]
#             yaw = np.round_(yaw_pitch_roll[0],5)

            f = open(label_output+str(label_idx+idx).zfill(6)+".txt", 'a')
            f.write(f'{x} {y} {z} {length} {width} {height} {yaw} {class_name}\n')
            f.close()
    return
    # return label_idx

                           

In [5]:
###calib, file movement exec###

asset_path_list = os.listdir(asset_path)
asset_path_list = natsort.natsorted(asset_path_list)


# print(asset_path_list)
pcd_idx=0
num_frame=[]
num_frame.append(0)
thres_flag=False
for data_cnt,path in enumerate(asset_path_list):
    
    files = os.listdir(asset_path+path)
    files = natsort.natsorted(files)
            
    for file in files:        
                   
        if '.bin' in file:
            #TODO : preprocession
            pcd_file_path = os.path.join(asset_path+path, file)
            copy_pcd('bin',pcd_file_path,output_path,pcd_idx)
            pcd_idx+=1
            # total_num_frame+=1
            
        if '.pcd' in file:
            pcd_file_path = os.path.join(asset_path+path, file)
            copy_pcd('pcd',pcd_file_path,output_path,pcd_idx)
            pcd_idx+=1
            # total_num_frame+=1
       
    if data_dir_num_thres <= data_cnt:
        break
    num_frame.append(pcd_idx)



In [6]:
# num_frame
pcd_idx

37318

In [7]:
# num_frame

In [8]:
#####label exec

label_list = os.listdir(label_path)
label_list = natsort.natsorted(label_list)

label_idx=0

preprocess_label(output_path,pcd_idx)



idx=0
for label_data_cnt,label in enumerate(label_list):
    print(idx)
    if asset_path_list[idx] == label.split('.')[0]:

        json_file_path = os.path.join(label_path, label)
        file = open(json_file_path)
        jsonString = json.load(file)
        convert_label_suite2custom(json_file_path,output_path, num_frame[idx])
        if data_dir_num_thres <= idx:
            break
        idx+=1
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
def delete_empty_asset(asset_path,empty_label_list):
#     label_path_list = os.listdir(label_path)
    empty_label_list = natsort.natsorted(empty_label_list)
    asset_path_list = os.listdir(asset_path)
    asset_path_list = natsort.natsorted(asset_path_list)
    empty_cnt=0
    print(empty_label_list)
    for asset in asset_path_list:
#         if empty_cnt == len(empty_label_list):
#             break
        if empty_cnt<len(empty_label_list) and asset.split('.')[0] == empty_label_list[empty_cnt] :
            os.remove(asset_path+asset)
            empty_cnt+=1
            continue
        
        os.rename(asset_path+asset, asset_path+str(int(asset.split('.')[0])-empty_cnt).zfill(6)+'.npy')
    print(empty_cnt)
  
    
    
   

In [10]:
def check_empty_label(label_path):
    label_path_list = os.listdir(label_path)
    label_path_list = natsort.natsorted(label_path_list)
    empty_num=0
    empty_list =[]
    for label in label_path_list:
        with open(label_path+label, 'r') as f:
            lines = f.readlines()
            if len(lines)==0:
                empty_num+=1
                empty_list.append(label.split('.')[0])
                os.remove(label_path+label)
                continue
        os.rename(label_path+label, label_path+str(int(label.split('.')[0])-empty_num).zfill(6)+'.txt')
    return empty_list

In [11]:
print(len(os.listdir(output_path+'labels/')))
print(len(os.listdir(output_path+'points/')))

37318
37318


In [12]:
empty_label_list = check_empty_label(output_path+'labels/')
delete_empty_asset(output_path+'points/',empty_label_list)
print(len(empty_label_list))

['000287', '000288', '000289', '000674', '000675', '000746', '000747', '000748', '000749', '000750', '000751', '000752', '000753', '000754', '000755', '000781', '000782', '000783', '000784', '000785', '000786', '000787', '000888', '000889', '000890', '000891', '000892', '000893', '000894', '000895', '000896', '000897', '000898', '000899', '000900', '000908', '000909', '000910', '000974', '000975', '000976', '000977', '000978', '000979', '000980', '000981', '000982', '000983', '000984', '000985', '000986', '000987', '001010', '001011', '001012', '001013', '001092', '001140', '001141', '001142', '001143', '001144', '001145', '001146', '002188', '002189', '002190', '002191', '002252', '002253', '002254', '002421', '002422', '002423', '002424', '002425', '003177', '003178', '003179', '003444', '003445', '003446', '003447', '004058', '004059', '004060', '004061', '004062', '004089', '004090', '004091', '004092', '004256', '004435', '004436', '004437', '004722', '004723', '004724', '004725',

In [13]:
print(len(os.listdir(output_path+'labels/')))
print(len(os.listdir(output_path+'points/')))

36681
36681


In [14]:
label_list = os.listdir(output_path+'labels/')
label_path_list = natsort.natsorted(label_list)
for idx,label in enumerate(label_path_list):
    if idx != int(label.split('.')[0]):
        print(idx)
        

In [15]:
asset_list = os.listdir(output_path+'points/')
asset_list = natsort.natsorted(asset_list)
for idx,asset in enumerate(asset_list):
    if idx != int(asset.split('.')[0]):
        print(idx)
        

In [16]:
#### split train, test
total_num=len(os.listdir(output_path+'points/'))

### shffle
total_num_list = list()
for i in range(total_num):
    total_num_list.append(i)
random.shuffle(total_num_list)

### mkdir
try:
    if not os.path.exists(output_path+'ImageSets/'):
        os.makedirs(output_path+'ImageSets/')
        print("generation")
except OSError:
    print("Error: Failed to create the directory.")
    
f2 = open(output_path+'ImageSets/'+'train'+".txt", 'w')
f3 = open(output_path+'ImageSets/'+'val'+".txt", 'w')
for idx,num in enumerate(total_num_list):
    if idx < total_num*train_ratio:
        f2.write(f'{str(num).zfill(6)}\n')
    else:
        f3.write(f'{str(num).zfill(6)}\n')
    idx+=1
f2.close()
f3.close()


generation


In [17]:
class_names

['Human',
 'Wheelchair',
 'Vehicle',
 'Stroller',
 'Pillar',
 'Two_Wheeled_Vehicle',
 'Traffic_Cone',
 'Shopping_Cart',
 'No_Parking_Sign',
 'Tow_Wheeled_Vehicle']

In [ ]:
def check_empty_label_asset(empty_label_list, pcd_idx):
    return True if pcd_idx in empty_label_list else False

In [ ]:
b=[1,2,3,4,5]
a=1
check_empty_label_asset(b,a)

In [ ]:
import numpy as np
n = len(b)
0.7*np.ones(shape=(n,n),dtype=np.float32)

In [ ]:
class_to_name={}
for i in range(n):
    class_to_name[i]='a'

In [ ]:
class_to_name